INTRODUCTION

This project aims to analyze the global impact of the COVID-19 pandemic on health outcomes and socioeconomic status. By examining datasets related to COVID-19 case numbers, deaths, vaccination rates, and socioeconomic indicators such as GDP, we will explore how the pandemic has affected different population groups worldwide. The goal is to identify patterns and provide insights that could inform public health policies and economic recovery efforts.

First Perspective:  
Countries with higher GDP and higher vaccination rates have managed the COVID-19 pandemic more effectively, resulting in lower mortality rates and better health outcomes despite high case numbers.

Arguments:  
Higher GDP Allows for Better Healthcare Infrastructure and Access to Medical Supplies:  
Visualization 1: GDP per capita vs. COVID-19 Deaths (scatter plot)  
Visualization 2: GDP per capita vs. Tests per Thousand (scatter plot)  
Higher Vaccination Rates Reduce Severe Cases and Deaths:  
Visualization 3: Mortality Rates and Vaccinations per 100 (map)  
Visualization 4: Excess Mortality vs. Vaccinations per 100 (scatter plot)

Second perspective:  
Lower-income countries faced greater difficulties in managing the COVID-19 pandemic due to limited healthcare resources and slower vaccine distribution, leading to higher mortality rates.

Arguments:  
Limited healthcare infrastructure and economic instability:  
Visualization 1: GDP vs. COVID-19 Deaths (scatter plot)  
Visualization 2: GDP vs. Tests per Thousand (scatter plot)  
Visualization 5: Mortality Rates and Case Numbers per Country (map)  
Visualization 6: Tests per Thousand vs. Total Cases (scatter Plot)  
Slower vaccine distribution:  
Visualization 3: Mortality Rates and Vaccinations per 100 (map)  
Visualization 4: Excess Mortality vs. Vaccinations per 100 (scatter plot)

Dataset and preprocessing  
The datasets that we use are the OWID Covid-19 dataset and the GDP per capita, PPP in US$ dataset. The Covid-19 dataset contains statistics on COVID-19 for every country, through the years 2020-2024. It has variables such as “Total_death” and “Total_cases”. The second dataset contains the GDP per capita in PPP in US$ per country per year. This means it contains the economic output in US dollars per inhabitant. PPP stands for purchasing power parity and it means the differences between countries have been normalized for differences in purchasing power, to make the comparisons more fair.  
The idea of the dataset is to give a reliable overview of the economic power of the countries per year.

The links for these datasets are:  
OWID Covid-19: [https://ourworldindata.org/coronavirus#deaths-and-cases-our-data-source](https://ourworldindata.org/coronavirus#deaths-and-cases-our-data-source)  
GDP per capita: [https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD](https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)

We preprocessed these databases by filtering it for the year 2021. Then for specific variable numbers we take the last value per country for 2021. For a few graphs we also filtered out some values in the location variable, we took out continents, the whole world and income categories. Because these values would make outliers in the graphs.


In [29]:
import plotly.express as px
import pandas as pd
import seaborn as sns

file_path = 'GDP-data.csv'
GDPdata = pd.read_csv(file_path, skiprows=4)
file_path = 'owid-covid-data.csv'
CovidData = pd.read_csv(file_path)

GDPdata = GDPdata.rename(columns={'Country Code': 'iso_code'})


CovidData['date'] = pd.to_datetime(CovidData['date'])
CovidData = CovidData[CovidData['date'] == '2020-12-31']

# Join de dataframes op de Date kolom
df = pd.merge(GDPdata, CovidData, on='iso_code', how='inner')

fig1 = px.choropleth(
    df, 
    locations="iso_code",
    color="total_cases_per_million",
    hover_name="Country Name",
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Cases per million by Country"
)

fig1.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    height=600
)

fig1.show()

fig2 = px.choropleth(
    df, 
    locations="iso_code",
    color="total_deaths_per_million",
    hover_name="Country Name",
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Deaths per million by Country"
)

fig2.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    height=600
)

fig2.show()

In [30]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import seaborn as sns

GDPdata = GDPdata.rename(columns={'Country Code': 'iso_code'})


CovidData['date'] = pd.to_datetime(CovidData['date'])
CovidData = CovidData[CovidData['date'] == '2020-12-31']

# Join de dataframes op de Date kolom
df = pd.merge(GDPdata, CovidData, on='iso_code', how='inner')
df = df.dropna(subset=['total_cases_per_million', 'total_deaths_per_million'])


fig = px.scatter(
    df,
    x="total_cases_per_million",
    y="total_deaths_per_million",
    hover_name="Country Name",
    trendline="ols",
    title="Comparison of Total Cases and Deaths per Million by Country",
    labels={
        "total_cases_per_million": "Total Cases per Million",
        "total_deaths_per_million": "Total Deaths per Million"
    }
)

correlation = df['total_cases_per_million'].corr(df['total_deaths_per_million'])
print(correlation)

# Update layout for better appearance
fig.update_traces(textposition='top center')
fig.update_layout(
    height=600
)

fig.show()

fig = px.scatter_geo(
    df, 
    locations="iso_code",
    size="total_cases_per_million",
    color="total_deaths_per_million",
    hover_name="Country Name",
    size_max=50,
    color_continuous_scale=px.colors.sequential.Reds,
    title="total cases and deaths per million by Country"
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    height=600
)

fig.show()

0.7246975031268591


In [53]:
CovidData = pd.read_csv('owid-covid-data.csv')
GDPdata = pd.read_csv('GDP-data.csv', skiprows=4)

Covid_2021 = CovidData[CovidData['date'].str.startswith('2021')]
Covid_deaths_2021 = Covid_2021.groupby('location').last()['total_deaths'].reset_index()

GDPdata = GDPdata.rename(columns={'Country Name': 'location'})

df = pd.merge(GDPdata, Covid_deaths_2021, on='location', how='inner')


df = df[df['location'] != 'World']
df = df[df['location'] != 'Upper middle income']
df = df[df['location'] != 'Lower middle income']
df = df[df['location'] !=  'High income']
df = df[df['location'] !=  'Low income']
df = df[df['location'] !=  'European Union']
df = df[df['location'] !=  'North America']
df = df[df['location'] !=  'South America']
df = df[df['location'] !=  'Asia']
df = df[df['location'] !=  'Oceania']
df = df[df['location'] !=  'Africa']

correlation = df['total_deaths'].corr(df['2021'])
print(correlation)

fig = px.scatter(
    df,
    x="2021",
    y="total_deaths",
    hover_name="location",
    trendline="ols",
    title="Comparison of GDP and Deaths per Million by Country",
    labels={
        "2021": "GDP",
        "total_deaths": "Total Deaths"
    }
)
fig

0.07710770419147137


In [46]:
covid_df = pd.read_csv('owid-covid-data.csv')
gdp_df = pd.read_csv('GDP-data.csv', skiprows=4)

# Filter data for 2021
covid_2021_df = covid_df[covid_df['date'].str.startswith('2021')]

# Convert 'date' column to datetime for proper sorting and comparison
covid_2021_df['date'] = pd.to_datetime(covid_2021_df['date'])

# Find the last available date for each location in 2021
last_dates_2021 = covid_2021_df.groupby('location')['date'].idxmax()

# Filter the dataframe to only include rows with the last available date for each location
covid_deaths_2021 = covid_2021_df.loc[last_dates_2021, ['location', 'total_deaths']]

# Reset the index
covid_deaths_2021 = covid_deaths_2021.reset_index(drop=True)

# Display the resulting dataframe
print(covid_deaths_2021)


              location  total_deaths
0          Afghanistan        7352.0
1               Africa      227755.0
2              Albania        3187.0
3              Algeria        6229.0
4       American Samoa           NaN
..                 ...           ...
249  Wallis and Futuna           7.0
250              World     5446891.0
251              Yemen        1983.0
252             Zambia        3697.0
253           Zimbabwe        4885.0

[254 rows x 2 columns]


/tmp/ipykernel_101809/3107810586.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

